In [ ]:
#!/usr/bin/env python

import rospy
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry
import time

y_p = 0;
y_s = 0;
t0 = time.time();
tp = t0;

def callback(msg, y_p, y_s, t0, tp):

    Kp=1;
    Kd=5;
    Ki=1;
    xm = msg.pose.pose.position.x
    ym = msg.pose.pose.position.y
    tm = time.time()
    if xm<3:  
        move_cmd.linear.x = 0.1
        cmd_vel.publish(move_cmd)
        if ym !=0:
            y_s = y_s + ym*(tm-tp)
            z_a = -(Kp*ym)-(Kd*(ym-y_p)/(tm-tp))-(Ki*y_s)
            move_cmd.linear.x = 0.1
            move_cmd.angular.z=z_a
            cmd_vel.publish(move_cmd)        
            y_p = ym
            tp = tm
            
            rospy.loginfo('x_position: {}'.format(xm))
            rospy.loginfo('y_position: {}'.format(ym))
        xm = msg.pose.pose.position.x
        ym = msg.pose.pose.position.y
        
    else:
        move_cmd.linear.x=0
        move_cmd.angular.z=0
        cmd_vel.publish(move_cmd)
    
if __name__ == '__main__':
    try:
        rospy.init_node('MoveForward_PID')
        move_cmd = Twist()
        cmd_vel = rospy.Publisher('/cmd_vel', Twist, queue_size=10)
        rospy.Subscriber("/odom", Odometry, callback, y_p, y_s, t0, tp)
        rospy.spin()
        
    except rospy.ROSInterruptException:
        rospy.loginfo("Node terimated")
